<a href="https://colab.research.google.com/github/alchemistcohen/Steam-Tables-App/blob/main/Steam_Tables_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steam tables contain specific volume, enthalpy, internal energy, and entropy of water (both liquid and vapor) over a range of temperatures and pressures, including at saturation and supercritical conditions.



1.   Saturation by Temperature: Input a temperature and fetch the saturation properties (e.g., pressure, specific volumes, enthalpy, and entropy).
2.   Saturation by Pressure: Input a pressure and fetch corresponding saturation properties.
3.   Interpolation/Extrapolation: Provide known values and a target to estimate properties.



In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr

Load the datasets ( Steam Tables)

In [ ]:
compressed_liquid_and_superheated_steam = pd.read_csv('/mnt/data/compressed_liquid_and_superheated_steam_V1.3.csv', skiprows=5, sep=',', engine='python')
saturated_by_pressure = pd.read_csv('/mnt/data/saturated_by_pressure_V1.4.csv', skiprows=5, sep=',', engine='python')
saturated_by_temperature = pd.read_csv('/mnt/data/saturated_by_temperature_V1.5.csv', skiprows=5, sep=',', engine='python')


In [ ]:
print(compressed_liquid_and_superheated_steam.head())  # Muestra las primeras 5 filas
print(saturated_by_pressure.head())
print(saturated_by_temperature.head())

   Pressure (MPa)   Temperature (°C)   Specific Volume (m^3/kg)  \
0            0.01                0.0                   0.001000   
1            0.01                5.0                   0.001000   
2            0.01               10.0                   0.001000   
3            0.01               15.0                   0.001001   
4            0.01               20.0                   0.001002   

   Density (kg/m^3)   Specific Internal Energy (kJ/kg)  \
0             999.8                          -0.040002   
1            999.92                          21.019999   
2            999.66                          42.019997   
3            999.06                          62.979991   
4            998.17                          83.909982   

   Specific Enthalpy (kJ/kg)  Specific Entropy [kJ/(kg K)]      Phase  
0                      -0.03                      -0.00015   "liquid"  
1                      21.03                       0.07625   "liquid"  
2                      42.03    

In [ ]:
columns_with_units = {
    "Compressed Liquid and Superheated Steam": [
        "Pressure (MPa)", "Temperature (°C)", "Specific Volume (m^3/kg)", "Density (kg/m^3)",
        "Specific Internal Energy (kJ/kg)", "Specific Enthalpy (kJ/kg)", "Specific Entropy [kJ/(kg K)]"
    ],
    "Saturated by Pressure": [
        "P (MPa)", "T (°C)", "Specific Volume Liquid (m^3/kg)", "Specific Volume Vapor (m^3/kg)",
        "Internal Energy Liquid (kJ/kg)", "Internal Energy Vapor (kJ/kg)", "Enthalpy Liquid (kJ/kg)",
        "Enthalpy Vapor (kJ/kg)", "Entropy Liquid [kJ/(kg K)]", "Entropy Vapor [kJ/(kg K)]"
    ],
    "Saturated by Temperature": [
        "T (°C)", "P (MPa)", "Specific Volume Liquid (m^3/kg)", "Specific Volume Vapor (m^3/kg)",
        "Internal Energy Liquid (kJ/kg)", "Internal Energy Vapor (kJ/kg)", "Enthalpy Liquid (kJ/kg)",
        "Enthalpy Vapor (kJ/kg)", "Entropy Liquid [kJ/(kg K)]", "Entropy Vapor [kJ/(kg K)]"
    ]
}

Function to interpolate data

In [ ]:
def interpolate_data(df, column, value):
    try:
        # Ensure the value is numeric
        value = float(value)
        df[column] = df[column].astype(float)

        # Sort and interpolate
        sorted_df = df.sort_values(by=column)
        interpolated_row = sorted_df.iloc[(sorted_df[column] - value).abs().argsort()[:2]]

        if len(interpolated_row) < 2:
            return "Not enough data points to interpolate."

        return interpolated_row.mean().to_dict()
    except Exception as e:
        return f"Error: {str(e)}"

Query function for Gradio

In [ ]:
def query_data(dataset, column_name, value):
    if dataset == "Compressed Liquid and Superheated Steam":
        df = compressed_liquid_and_superheated_steam
    elif dataset == "Saturated by Pressure":
        df = saturated_by_pressure
    elif dataset == "Saturated by Temperature":
        df = saturated_by_temperature
    else:
        return "Invalid dataset selected."


    if column_name not in df.columns:
        return f"Column '{column_name}' not found in the dataset."


    return interpolate_data(df, column_name, value)


with gr.Blocks() as gr_app:
    gr.Markdown("## Steam Tables ")

    with gr.Row():
        dataset_dropdown = gr.Dropdown(
            choices=[
                "Compressed Liquid and Superheated Steam",
                "Saturated by Pressure",
                "Saturated by Temperature"
            ],
            label="Select Dataset"
        )
        column_dropdown = gr.Dropdown(
            choices=[],
            label="Select Column (with Units)",
            interactive=True
        )
        value_input = gr.Textbox(label="Value", placeholder="Enter the value to query (numeric)")

    output = gr.JSON(label="Interpolated Data")


    def update_columns(dataset):
        return gr.update(choices=columns_with_units.get(dataset, []))

    dataset_dropdown.change(update_columns, inputs=dataset_dropdown, outputs=column_dropdown)

    submit_button = gr.Button("Query Data")
    submit_button.click(query_data, inputs=[dataset_dropdown, column_dropdown, value_input], outputs=output)

In [ ]:
gr_app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8af90a0892846bccc2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
